# Optimizar espacio para trabajar con DataFrames

Cuando importamos datos de un fichero y los cargamos en un DataFrame, debemos de estar atentos a cómo se guarda la información ya que podría estar usándose más espacio en memoria del necesario.

In [3]:
import pandas as pd
df = pd.read_csv("data/datosAImportarConFechas.csv")

In [4]:
#Exploramos los datos cargados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Dia             100000 non-null  object 
 1   Mes             100000 non-null  object 
 2   Cantidad        100000 non-null  float64
 3   Candidatos      100000 non-null  int64  
 4   Tiempo          100000 non-null  object 
 5   FechaPropuesta  100000 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.6+ MB


## Número de filas y Columnas

In [5]:
numfilinicial = len(df)
numcolinicial = len(df.columns)

In [6]:
numfilinicial

100000

In [7]:
numcolinicial

6

memory_usage="deep"
Haciendo uso del atributo memory_usage="deep", podemos obtener una información más precisa sobre el espacio que ocupa un DataFrame.

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Dia             100000 non-null  object 
 1   Mes             100000 non-null  object 
 2   Cantidad        100000 non-null  float64
 3   Candidatos      100000 non-null  int64  
 4   Tiempo          100000 non-null  object 
 5   FechaPropuesta  100000 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 26.6 MB


Vemos que las columnas de tipo object son las que más ocupan con diferencia. 

## Vamos a ver cómo optimizar el uso de memoria de cada tipo paso a paso.

In [9]:
df.head()

,Dia,Mes,Cantidad,Candidatos,Tiempo,FechaPropuesta
0,Domingo,Abril,151.148659,15,nieve,2010-4-4
1,Jueves,Agosto,459.638424,13,nieve,2010-4-18
2,Jueves,Marzo,519.621128,15,luvia,2010-12-12
3,Domingo,Julio,52.883518,8,soleado,2010-11-13
4,Domingo,Octubre,497.774052,8,nublado,2010-4-9


### Seleccionar un tipo de columna

In [73]:
datos = df.select_dtypes(include="number")
datos

,Cantidad,Candidatos
0,151.148659,15
1,459.638424,13
2,519.621128,15
3,52.883518,8
4,497.774052,8
...,...,...
99995,375.582302,7
99996,544.626700,18
99997,866.428214,4
99998,492.126995,12


### Num - Optimizando el espacio que ocupan las columnas de números

Tenemos una sola columna de tipo entero, donde podemos observar que se almacenan los números usando 64 bits. Con esta cantidad de bits, podemos representar números muy grandes que quizás no estemos utilizando. Vamos a ver cómo son los datos que almacenamos

In [10]:
df["Candidatos"].describe()

count    100000.000000
mean         10.509630
std           5.772111
min           1.000000
25%           5.000000
50%          11.000000
75%          16.000000
max          20.000000
Name: Candidatos, dtype: float64

Observamos que tenemos números entre 1 y 20. Por tanto, podemos representarlos usando menos información. Para ello, usamos el método to_numeric de pandas y el atributo downcast="unsigned" para especificar números sin signo (ya que todos son positivos). Convertimos las columnas que contienen enteros y vemos el resultado.

El método to_numeric con el atributo downcast permite almacenar un número en el tipo más pequeño posible.

Si tuviésemos números positivos y negativos, usaríamos la opción integer.

In [18]:
df2 = df.copy()
df2.columns

Index(['Dia', 'Mes', 'Cantidad', 'Candidatos', 'Tiempo', 'FechaPropuesta'], dtype='object')

In [19]:
str(df2['Candidatos'].dtype)

'int64'

In [20]:
#reducimos espacio de las columnas de tipo entero
df2 = df.copy()
for col in df2.columns:            
       if "int" in str(df2[col].dtype):
            df2[col] = pd.to_numeric(df2[col],downcast='unsigned')

In [21]:
df2.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Dia             100000 non-null  object 
 1   Mes             100000 non-null  object 
 2   Cantidad        100000 non-null  float64
 3   Candidatos      100000 non-null  uint8  
 4   Tiempo          100000 non-null  object 
 5   FechaPropuesta  100000 non-null  object 
dtypes: float64(1), object(4), uint8(1)
memory usage: 26.2 MB


Realizamos el mismo proceso con los números en coma flotante usando el valor float para el atributo downcast

In [26]:
#reducimos espacio de las columnas de tipo float
df3 = df2.copy()
for col in df3.columns:            
       if "float" in str(df3[col].dtype):
            df3[col] = pd.to_numeric(df3[col],downcast='float')

In [30]:
df3.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Dia             100000 non-null  object 
 1   Mes             100000 non-null  object 
 2   Cantidad        100000 non-null  float32
 3   Candidatos      100000 non-null  uint8  
 4   Tiempo          100000 non-null  object 
 5   FechaPropuesta  100000 non-null  object 
dtypes: float32(1), object(4), uint8(1)
memory usage: 26.6 MB


## OBJ - Reducir espacio del tipo object

Tenemos que explorar el tipo para ver cómo podemos reducirlo. Si nos fijamos, está usándose texto para representar una serie de categorías. De hecho, a través de unique podemos ver cuántas categorías distintas tenemos

In [31]:
df3["Mes"].unique()

array(['Abril', 'Agosto', 'Marzo', 'Julio', 'Octubre', 'Enero',
       'Diciembre', 'Septiembre', 'Mayo', 'Noviembre', 'Junio', 'Febrero'],
      dtype=object)

In [32]:
df3["Dia"].unique()

array(['Domingo', 'Jueves', 'Viernes', 'Martes', 'Miércoles', 'Lunes',
       'Sábado'], dtype=object)

In [33]:
df3["Tiempo"].unique()

array(['nieve', 'luvia', 'soleado', 'nublado'], dtype=object)

En los tres casos tenemos como mucho 12 valores distintos por categoría. Sería más eficiente guardar la información asociada a la categoría en lugar del texto, que ocupa mucho más. Usando categorías, una columna puede ocupar el mismo espacio que si usásemos enteros. 

Probemos a convertir las columnas de tipo objeto a categoría.

In [34]:
df4 = df3.copy()
for col in df4.columns: 
    if col != "FechaPropuesta":
       if "object" in str(df4[col].dtype):
            df4[col] = df4[col].astype('category')

In [36]:
df4.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Dia             100000 non-null  category
 1   Mes             100000 non-null  category
 2   Cantidad        100000 non-null  float32 
 3   Candidatos      100000 non-null  uint8   
 4   Tiempo          100000 non-null  category
 5   FechaPropuesta  100000 non-null  object  
dtypes: category(3), float32(1), object(1), uint8(1)
memory usage: 7.8 MB


### Fechas

In [37]:
df4["FechaPropuesta"] = pd.to_datetime(df4["FechaPropuesta"])
df = df4.copy()

In [38]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Dia             100000 non-null  category      
 1   Mes             100000 non-null  category      
 2   Cantidad        100000 non-null  float32       
 3   Candidatos      100000 non-null  uint8         
 4   Tiempo          100000 non-null  category      
 5   FechaPropuesta  100000 non-null  datetime64[ns]
dtypes: category(3), datetime64[ns](1), float32(1), uint8(1)
memory usage: 2.3 MB


Calcular la longitud y numeros de columnas finales

In [39]:
numfilfinal = len(df)
numcolfinal = len(df.columns)

dif_filas = numfilinicial - numfilfinal
dif_col = numcolinicial - numcolfinal 

In [40]:
dif_filas

0

In [41]:
dif_col 

0

Hemos reducido el uso de memoria de 26.6 MB a 2.3 MB